## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1599 entries, 1 to 1599
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed.acidity         1599 non-null   float64
 1   volatile.acidity      1599 non-null   float64
 2   citric.acid           1599 non-null   float64
 3   residual.sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free.sulfur.dioxide   1599 non-null   float64
 6   total.sulfur.dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 162.4 KB


Convert quality to a binary variable, with the dividing line between 5 and 6.

In [4]:
wine.quality.value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [5]:
# answer below
wine['High_Quality'] = np.where(wine['quality'] >= 5, 1, 0)
clean_wine = wine.drop(columns='quality')
clean_wine.head(3)
clean_wine.High_Quality.value_counts()

1    1536
0      63
Name: High_Quality, dtype: int64

Create a train test split with 20% of the data in the test subsample.

In [6]:
# answer below:
from sklearn.model_selection import train_test_split

y = clean_wine.High_Quality
X = clean_wine.drop(columns='High_Quality')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.20)

Scale only the independent variables using the minmax scaler.

In [7]:
# answer below:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [8]:
# answer below:
from sklearn.neighbors import KNeighborsClassifier

neighbors = KNeighborsClassifier(n_neighbors= 5)

neighbors.fit(X_train, y_train)

print('Train Score: {:.4f}'.format(neighbors.score(X_train, y_train)))
print('Test Score: {:.4f}'.format(neighbors.score(X_test, y_test)))

Train Score: 0.9672
Test Score: 0.9437


In [9]:
neighbors.fit(X_train_scaled, y_train)

print('Train Score: {:.4f}'.format(neighbors.score(X_train_scaled, y_train)))
print('Test Score: {:.4f}'.format(neighbors.score(X_test_scaled, y_test)))

# When the sets are scaled, we don't see as much underfitting

Train Score: 0.9664
Test Score: 0.9500


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [10]:
# answer below:
# weight of 1/sqrt(distance)

def new_weight(dist):
  with np.errstate(divide='ignore'):
    dist = 1. / np.sqrt(dist)
  inf_mask = np.isinf(dist)
  inf_row = np.any(inf_mask, axis=1)
  dist[inf_row] = inf_mask[inf_row]
  return dist

neighbors = KNeighborsClassifier(n_neighbors= 5, weights=new_weight)

neighbors.fit(X_train, y_train)

print('Train Score: {:.4f}'.format(neighbors.score(X_train, y_train)))
print('Test Score: {:.4f}'.format(neighbors.score(X_test, y_test)))

Train Score: 1.0000
Test Score: 0.9437


In [11]:
neighbors = KNeighborsClassifier(n_neighbors= 5, weights=new_weight)

neighbors.fit(X_train_scaled, y_train)

print('Train Score: {:.4f}'.format(neighbors.score(X_train_scaled, y_train)))
print('Test Score: {:.4f}'.format(neighbors.score(X_test_scaled, y_test)))

Train Score: 1.0000
Test Score: 0.9500
